In [1]:
#Text Alert Service Initialization

from ipaddress import IPv4Address  # for your IP address
from pyairmore.request import AirmoreSession  # to create an AirmoreSession
from pyairmore.services.messaging import MessagingService  # to send messages

currip="192.168.0.108" #Put IP inside the quotes


ip = IPv4Address(currip)  #Create an IP address object
#Create a session
session = AirmoreSession(ip)
# if your port is not 2333 --> session = AirmoreSession(ip, 2334)  [assuming it is 2334]

print("Server Running: ", session.is_server_running)
was_accepted = session.request_authorization()
print("Session Authorization: ",was_accepted)

service = MessagingService(session) #Creating the service


ModuleNotFoundError: No module named 'pyairmore'

In [2]:
#tweak frames and padding of boxes -Global var
frames = 5 # lower value gives more number of frames, 30 is default

#Library Imports
#import argparse Not Used
import numpy as np 
import time
import cv2
import os
import matplotlib.pyplot as plt


def obj(name):
    value = name
    global last_detection
    
    # load the COCO class labels our YOLO model was trained on
    labelsPath = os.path.sep.join(['yolo-coco', "coco.names"])
    LABELS = open(labelsPath).read().strip().split("\n")

    # initialize a list of colors to represent each possible class label
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
        dtype="uint8")

    # derive the paths to the YOLO weights and model configuration
    weightsPath = os.path.sep.join(['yolo-coco', "yolov3.weights"])
    configPath = os.path.sep.join(['yolo-coco', "yolov3.cfg"])

    # load our YOLO object detector trained on COCO dataset (80 classes)
    print("[INFO] loading YOLO from disk...")
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

    # load our input image and grab its spatial dimensions
    image = cv2.imread(value)
    (H, W) = image.shape[:2]

    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    # show timing information on YOLO
    print("[INFO] YOLO took {:.6f} seconds".format(end - start))

    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > 0.5:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping bounding
    # boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,
        0.3)
   
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            temp = []
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the image
            color = [int(c) for c in COLORS[classIDs[i]]]
            

            
            #text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            text = "{}".format(LABELS[classIDs[i]])
            #print(text)
            if text == "person":
                curr_time=time.time()
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                #cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                cv2.putText(image, str(i)+", "+str(x)+" "+(str(y)), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
                if curr_time-last_detection>5:
                    service.send_message("9449277201", "Intruder Alert !") #Put phone number of receiver in first set of quotes
                last_detection=time.time()
    
                cv2.imwrite("test/detected/time{}.jpg".format(curr_time),image)
    
    
#Program Execution Starts
# print("Enter Video Path")
# path = input()
cap = cv2.VideoCapture(0) 
# cap = cv2.VideoCapture(path)
j = 0
i=0
last_detection=0

while True:
    ret, frame = cap.read()
    
    if ret: # Change Made 
    
        cv2.imshow('frame', frame)   #print(frame)
        frame = cv2.resize(frame,(400,300))
        if i%frames == 0:
            j = j+1
            s = "test/ss{}.jpg".format(j)
            cv2.imwrite(s,frame)
            obj(s)
            
        i=i+1



        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()


[INFO] loading YOLO from disk...
[INFO] YOLO took 2.641910 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.999281 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.996734 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.907026 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.947060 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 1.119602 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.978873 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 1.278972 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 1.400792 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 1.047912 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.966035 seconds
[INFO] loading YOLO from disk...
[INFO] YOLO took 1.008204 seconds
